In [ ]:
## Set notebook to auto reload updated modules
%load_ext autoreload
%autoreload 2

In [ ]:
NB_LOG_LEVEL: str = "ERROR"

In [ ]:
import json
from pathlib import Path
import typing as t
from dataclasses import dataclass, field

In [ ]:
import core_utils, setup, settings
from cfapi.controllers import CloudflareController, get_cloudflare_controller

In [ ]:
import pandas as pd

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
LOGGING_INITIALIZED: bool = False

if not LOGGING_INITIALIZED:
    setup.setup_loguru_logging(log_level=NB_LOG_LEVEL)
    LOGGING_INITIALIZED = True

In [ ]:
@dataclass
class CloudflareConfig(object):
    cf_api_email: str | None 
    cf_api_key: str | None = field(default=None, repr=False)
    cf_api_token: str | None = field(default=None, repr=False)
    
    @property
    def token(self):
        return self.cf_api_token
    
    @property
    def key(self):
        return self.cf_api_key
    
    @property
    def email(self):
        return self.cf_api_email


@dataclass
class NotebookConfig(object):
    cf_config: CloudflareConfig = None

In [ ]:
def load_nb_config(config_file: str = "./nb_config.json") -> NotebookConfig:
    if not config_file:
        raise ValueError("Missing path to a notebook config JSON file.")
    
    config_file: Path = Path(config_file).expanduser() if "~" in str(config_file) else Path(str(config_file)).absolute()
    
    if not config_file.exists():
        raise FileNotFoundError(f"Could not find notebook config file at '{config_file}'")
    
    display(f"Loading notebook config from: {config_file}")
    with open(config_file, "r") as f:
        nb_config_dict: dict = json.load(f)
    
    cf_config: CloudflareConfig = CloudflareConfig(**nb_config_dict["cloudflare"])
    nb_config: NotebookConfig = NotebookConfig(cf_config=cf_config)
    
    return nb_config

In [ ]:
nb_config = load_nb_config()
if nb_config:
    display(nb_config)

In [ ]:
api_token = nb_config.cf_config.token
if not api_token:
    raise ValueError("Missing Cloudflare API token. Set environment variable CF_API_TOKEN=<API_TOKEN>")

display(api_token)


In [ ]:
cf_controller = CloudflareController(debug_secrets=True, api_token=api_token)

In [ ]:
accounts = cf_controller.get_accounts()
accounts

In [ ]:
accounts_df = pd.DataFrame(accounts)
accounts_df

In [ ]:
zones = cf_controller.get_zones()
zones

In [ ]:
zones_df = pd.DataFrame(zones)
zones_df

In [ ]:
working_zone = zones_df[zones_df["name"] == "crvr.us"].iloc[0]
working_zone

In [ ]:
display(working_zone["name"])
display(working_zone["id"])

In [ ]:
working_zone_waf_filters = cf_controller.get_zone_waf_filters(zone_id=working_zone['id'])
working_zone_waf_filters

In [ ]:
working_zone_waf_filters_df = pd.DataFrame(working_zone_waf_filters)
working_zone_waf_filters_df

In [ ]:
zones_wafs = []

for zone in zones:
    try:
        waf_rules = cf_controller.get_zone_waf_filters(zone_id=zone["id"])
    except Exception as exc:
        msg = f"({type(exc)}) Error getting Cloudflare zone WAF rules. Details: {exc}"
        display(f"[ERROR] {msg}")

        continue
    # zones_wafs.append({"zone_name": zone.name, "zone_id": zone.id, "waf_rules": waf_rules})
    for rule in waf_rules:
        zones_wafs.append(
            {
                "zone_name": zone["name"],
                "zone_id": zone["id"],
                "zone_status": zone["status"],
                "zone_paused": zone["paused"],
                "waf_rule_id": rule["id"],
                "waf_rule": rule["expression"],
                "waf_rule_paused": rule["paused"],
            }
        )

zones_wafs_df = pd.DataFrame(zones_wafs)
zones_wafs_df